## Data uploading

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten,
                                     Activation, Dense, Dropout,
                                     BatchNormalization)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
import os
import shutil
import itertools
import pathlib
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import os
import pandas as pd

train_data_path = '/Users/nikitarodionov/Downloads/archive/Training'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(train_data_path)
         if os.path.isdir(os.path.join(train_data_path, fold))]

for fold in folds:
    f_path = os.path.join(train_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
train_df = pd.concat([Fseries, Lseries], axis=1)
train_df

In [ ]:
test_data_path = '/Users/nikitarodionov/Downloads/archive/Testing'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(test_data_path)
         if os.path.isdir(os.path.join(test_data_path, fold))]

for fold in folds:
    f_path = os.path.join(test_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
test_df = pd.concat([Fseries, Lseries], axis=1)
test_df

## TensorFlow Image Data Generator and classes dataframes

In [ ]:
img_size = (224, 244)
batch_size = 16
valid, test = train_test_split(test_df, train_size=0.5,
                               shuffle=True, random_state=42)
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df, x_col='filepaths',
                                       y_col='label', target_size=img_size,
                                       class_mode='categorical',
                                       color_mode='rgb', shuffle=True,
                                       batch_size=batch_size)

valid_gen = ts_gen.flow_from_dataframe(valid, x_col='filepaths',
                                       y_col='label', target_size=img_size,
                                       class_mode='categorical',
                                       color_mode='rgb', shuffle=True,
                                       batch_size=batch_size)

test_gen = ts_gen.flow_from_dataframe(test, x_col='filepaths', y_col='label',
                                      target_size=img_size,
                                      class_mode='categorical',
                                      color_mode='rgb',
                                      shuffle=False, batch_size=batch_size)
# Classes dataframes
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images, labels = next(train_gen)

## Model Structure

In [ ]:
img_shape = (img_size[0], img_size[1], 3)
num_class = len(classes)

base_model = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=img_shape,
    pooling='max')
model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(l=0.01),
          activity_regularizer=regularizers.l1(0.005),
          bias_regularizer=regularizers.l1(0.005), activation='relu'),
    Dropout(rate=0.4, seed=75),
    Dense(num_class, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy',
              metrics=['accuracy'])

## Model main info

In [ ]:
model.summary()

## Iterative training

In [ ]:
Epochs = 2

history = model.fit(x=train_gen, epochs=Epochs,
                    verbose=1, validation_data=valid_gen,
                    validation_steps=None, shuffle=False)

In [ ]:
# Load the single image
image = tf.keras.utils.load_img(
    '/Users/nikitarodionov/Downloads/archive/Testing/notumor/Te-no_0404.jpg',
    target_size=(img_size[0], img_size[1]))
image = tf.keras.utils.img_to_array(image)
image = tf.expand_dims(image, axis=0)
image = tf.keras.applications.efficientnet.preprocess_input(image)
predictions = model.predict(image)
predicted_class = tf.math.argmax(predictions, axis=1)
class_dict_local = {0 : 'glioma', 1 : 'meningioma', 2: 'notumor', 3 : 'pituitary'}
predicted_class_literal = class_dict_local[predicted_class.numpy()[0]]
print('The predicted class is:', predicted_class_literal)

## Common Metrics

In [ ]:
train_score = model.evaluate(train_gen, steps=16, verbose=1)
valid_score = model.evaluate(valid_gen, steps=16, verbose=1)
test_score = model.evaluate(test_gen, steps=16, verbose=1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

## Metrics of classes

In [ ]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(classification_report(test_gen.classes, y_pred, target_names=classes))